# Tier 1: probing each target prefixes

In [1]:
import logging
import pickle
import json

from pathlib import Path
import uuid

from geoloc_imc_2023.cbg import CBG, get_prefix_from_ip
from geoloc_imc_2023.query_api import get_measurements_from_tag

from geoloc_imc_2023.default import (
    PROBES_PATH,
    ANCHORS_PATH,
    HITLIST_PATH,
    RIPE_CREDENTIALS,
)

logging.basicConfig(level=logging.INFO)

NB_TARGET = 2
NB_VP = 5

## load datasets

In [2]:
# load hitlist
with open(HITLIST_PATH, "rb") as f:
    targets_per_prefix = pickle.load(f)

# load probes
with open(PROBES_PATH, "rb") as f:
    probes = pickle.load(f)

# load anchors
with open(ANCHORS_PATH, "rb") as f:
    anchors = pickle.load(f)

## select target and vps dataset

In [3]:
# select targets and vps from anchors
targets= list(anchors.keys())[:NB_TARGET]
vps = {}
for i, probe in enumerate(probes):
    if i >= NB_VP: break
    vps[probe] = probes[probe]

logging.info(f"nb targets: {len(targets)}")
logging.info(f"nb_vps : {len(vps)}")

INFO:root:nb targets: 3
INFO:root:nb_vps : 10


## measure target prefixes

In [4]:
dry_run = True
measurement_uuid = uuid.uuid4()

# measurement configuration for retrieval
measurement_config = {
    "UUID": str(measurement_uuid),
    "is_dry_run": dry_run,
    "description": "measurement towards all anchors with all probes as vps",
    "type" : "target prefix",
    "af" : 4,
    "targets": targets,
    "vps": vps,
}

# save measurement configuration before starting measurement
measurement_config_path = Path(".") / f"../measurements_metadata/{str(measurement_uuid)}.json"
with open(measurement_config_path, "w") as f:
    json.dump(measurement_config, f, indent=4)

cbg = CBG(RIPE_CREDENTIALS)

# get target prefixes
target_prefixes = []
for target_addr in targets:
    target_prefix = get_prefix_from_ip(target_addr)
    target_prefixes.append(target_prefix)

# measurement for 3 targets in every target prefixes
measurement_config["start_time"], measurement_config["end_time"] = cbg.prefix_probing(
    target_prefixes= target_prefixes, 
    vps= vps,
    targets_per_prefix= targets_per_prefix,
    tag= measurement_uuid,
    dry_run=dry_run
)

# save measurement configuration before starting measurement
measurement_config_path = Path(".") / f"../measurements_metadata/{str(measurement_uuid)}.json"
with open(measurement_config_path, "w") as f:
    json.dump(measurement_config, f, indent=4)

INFO:root:starting measurement for target_prefix='213.225.160.0' with ['213.225.160.159', '213.225.160.140', '213.225.160.14'] with 10 vps
INFO:root:starting measurement for target_prefix='145.220.0.0' with ['145.220.0.1', '145.220.0.28', '145.220.0.29'] with 10 vps
INFO:root:Reached limit for number of conccurent measurements: 5 (limit is 5)
INFO:root:starting measurement for target_prefix='5.28.0.0' with ['5.28.0.6', '5.28.0.13', '5.28.0.14'] with 10 vps
INFO:root:measurement : e6000fd2-d187-4947-99ed-8b1cf36f2d8c done


## Retrieve prefix probing results 

In [ ]:
measurement_results = get_measurements_from_tag(measurement_config["UUID"])

# test that everything is alright
logging.info(f"nb measurements retrieved: {len(measurement_results)}")

print(f"measurements for {len(measurement_results)} target addr.")
for i, (target_addr, results) in enumerate(measurement_results.items()):
    if i > 10: break
    print(target_addr, ":", len(results))

# save results
out_file = f"../results/{measurement_config['UUID']}.pickle"
print(out_file)
with open(out_file, "wb") as f:
    pickle.dump(measurement_results,f)
    

# Tier 2: probe each target

In [2]:
import logging
import pickle
import json
import uuid

from pathlib import Path
from random import choice

from geoloc_imc_2023.query_api import get_measurements_from_tag, get_measurement_from_id
from geoloc_imc_2023.cbg import CBG
from geoloc_imc_2023.default import (
    PROBES_PATH,
    ANCHORS_PATH,
    RIPE_CREDENTIALS,
)

logging.basicConfig(level=logging.DEBUG)


NB_TARGET = 2
NB_VP = 5

## load data

In [27]:
# load probes
with open(PROBES_PATH, "rb") as f:
    probes = pickle.load(f)

# load anchors
with open(ANCHORS_PATH, "rb") as f:
    anchors = pickle.load(f)

## get targets and vps datasets

In [29]:
# select targets and vps from anchors
targets= set()
while len(targets) < NB_TARGET:
    random_target = choice(list(anchors))
    targets.add(random_target)
target = list(targets)

# select vps from probes
vps = {}
while len(vps) < NB_VP:
    random_vp = choice(list(probes))
    vps[random_vp] = probes[random_vp]

logging.info(f"nb targets: {len(targets)}")
logging.info(f"nb_vps : {len(vps)}")

INFO:root:nb targets: 2
INFO:root:nb_vps : 5


## measure targets

In [6]:
dry_run = True
measurement_uuid = uuid.uuid4()

# measurement configuration for retrieval
measurement_config = {
    "UUID": str(measurement_uuid),
    "is_dry_run": dry_run,
    "description": "measurement towards all anchors with all probes as vps",
    "type" : "target prefix",
    "af" : 4,
    "targets": targets,
    "vps": vps,
}

# save measurement configuration before starting measurement
measurement_config_path = Path(".") / f"../measurements_metadata/{str(measurement_uuid)}.json"
with open(measurement_config_path, "w") as f:
    json.dump(measurement_config, f)

cbg = CBG(RIPE_CREDENTIALS)

# measurement for 3 targets in every target prefixes
measurement_config["ids"], measurement_config["start_time"], measurement_config["end_time"] = cbg.target_probing(
    targets= targets, 
    vps= vps,
    tag= measurement_uuid,
    dry_run=dry_run
)

DEBUG:root:starting measurement for target_addr='217.70.176.109' with 5 vps
INFO:root:from atlas217.70.176.109
INFO:root:from atlas[55328, 55212, 1000927, 22312, 61638]
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): atlas.ripe.net:443
DEBUG:urllib3.connectionpool:https://atlas.ripe.net:443 "POST /api/v2/measurements/?key=b3d3d4fc-724e-4505-befe-1ad16a70dc87 HTTP/1.1" 201 27
DEBUG:root:starting measurement for target_addr='130.104.228.159' with 5 vps
INFO:root:from atlas130.104.228.159
INFO:root:from atlas[55328, 55212, 1000927, 22312, 61638]
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): atlas.ripe.net:443
DEBUG:urllib3.connectionpool:https://atlas.ripe.net:443 "POST /api/v2/measurements/?key=b3d3d4fc-724e-4505-befe-1ad16a70dc87 HTTP/1.1" 201 27
INFO:root:measurement : 7e273b92-04e3-44e3-946f-3337f9e38637 done


In [7]:
measurement_config_path = Path(".") / f"../measurements_metadata/{str(measurement_uuid)}.json"
with open(measurement_config_path, "w") as f:
    json.dump(measurement_config, f, indent=4)

## retrieve measurement results

In [3]:
from geoloc_imc_2023.query_api import get_measurements_from_tag, get_measurement_url
import time
import requests
import logging

def get_from_atlas(url: str, max_retry: int = 60):
    """request to Atlas API"""

    for _ in range(max_retry):
        response = requests.get(url, timeout=20).json()
        print(response)
        if response:
            break
        time.sleep(2)

    return response

measurement_id = 52712915
url = f"https://atlas.ripe.net/api/v2/measurements/{measurement_id}/"
print(url)
response = requests.get(url, timeout=20).json()
logging.info(f"RESP: {response['status']}")
logging.info(f"RESP: {response['result']}")

result = requests.get(response['result'])
print(result.content)


https://atlas.ripe.net/api/v2/measurements/52712915/
b'[{"fw":5080,"mver":"2.6.2","lts":11,"dst_name":"217.70.176.109","af":4,"dst_addr":"217.70.176.109","src_addr":"10.71.71.15","proto":"ICMP","ttl":50,"size":48,"result":[{"rtt":167.635967},{"rtt":167.593986},{"rtt":167.989166}],"dup":0,"rcvd":3,"sent":3,"min":167.593986,"max":167.989166,"avg":167.7397063333,"msm_id":52712915,"prb_id":1000927,"timestamp":1682590165,"msm_name":"Ping","from":"103.38.202.251","type":"ping","group_id":52712915,"step":null,"stored_timestamp":1682598256},{"fw":5080,"mver":"2.6.2","lts":6,"dst_name":"217.70.176.109","af":4,"dst_addr":"217.70.176.109","src_addr":"192.168.0.46","proto":"ICMP","ttl":53,"size":48,"result":[{"rtt":90.787459},{"rtt":89.425583},{"rtt":91.57775}],"dup":0,"rcvd":3,"sent":3,"min":89.425583,"max":91.57775,"avg":90.5969306667,"msm_id":52712915,"prb_id":55212,"timestamp":1682590166,"msm_name":"Ping","from":"64.65.80.109","type":"ping","group_id":52712915,"step":null,"stored_timestamp":16

In [8]:
measurement_results = get_measurements_from_tag(measurement_config["UUID"])

if not measurement_results:
    for measurement_id in measurement_config["ids"]: 
        logging.info(f"getting measurement with id : {measurement_id}")
        measurement_result = get_measurement_from_id(measurement_id)

logging.info(f"measurements for {len(measurement_results)} target addr")
for i, (target_addr, results) in enumerate(measurement_results.items()):
    if i > 10: break
    logging.info(f" {target_addr} : number of measurements {len(results)}")

# save results
out_file = f"../results/{measurement_config['UUID']}.pickle"
logging.info(out_file)
with open(out_file, "wb") as f:
    pickle.dump(measurement_results,f)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): atlas.ripe.net:443
DEBUG:urllib3.connectionpool:https://atlas.ripe.net:443 "GET /api/v2/measurements/tags/3e223905-d775-4c7a-88a8-8be9bdb21d6e/results/?key=b3d3d4fc-724e-4505-befe-1ad16a70dc87 HTTP/1.1" 200 None
INFO:root:measurements for 3 target addr
INFO:root: 213.225.160.239 : number of measurements 7
INFO:root: 145.220.0.55 : number of measurements 7
INFO:root: 5.28.0.17 : number of measurements 7
INFO:root:../results/b59a0873-574d-46bd-81c8-860ba3733be4.pickle


defaultdict(<class 'list'>, {'213.225.160.239': [{'node': '45.138.229.91', 'min_rtt': 22.373933, 'rtt_list': [23.420751, 22.415617, 22.373933]}, {'node': '193.0.0.78', 'min_rtt': 11.771138, 'rtt_list': [12.25421, 12.122115, 11.771138]}, {'node': '83.81.82.33', 'min_rtt': 29.290758, 'rtt_list': [31.130887, 30.42663, 29.290758]}, {'node': '87.3.187.45', 'min_rtt': 37.307144, 'rtt_list': [38.498312, 37.307144, 37.986824]}, {'node': '77.174.30.45', 'min_rtt': 33.076999, 'rtt_list': [33.938695, 33.787399, 33.076999]}, {'node': '93.108.63.51', 'min_rtt': 39.656456, 'rtt_list': [40.313097, 40.373769, 39.656456]}, {'node': '216.147.126.54', 'min_rtt': 187.236902, 'rtt_list': [191.851814, 187.236902, 200.42884]}], '145.220.0.55': [{'node': '193.0.0.78', 'min_rtt': 2.353152, 'rtt_list': [7.880449, 2.416129, 2.353152]}, {'node': '45.138.229.91', 'min_rtt': 7.16243, 'rtt_list': [8.023471, 7.310364, 7.16243]}, {'node': '83.81.82.33', 'min_rtt': 17.528067, 'rtt_list': [17.528067, 31.00647, 18.276099